## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-26-13-08-59 +0000,nypost,Travel woes remain across NYC over Winter Stor...,https://nypost.com/2026/01/26/us-news/travel-w...
1,2026-01-26-13-06-14 +0000,nyt,Gold Soars to Record High as Global Tensions Grow,https://www.nytimes.com/2026/01/26/business/go...
2,2026-01-26-13-04-55 +0000,nyt,Gunmen Kill 11 After Soccer Match in Mexico,https://www.nytimes.com/2026/01/26/world/ameri...
3,2026-01-26-13-04-50 +0000,nypost,Dems need to ‘tell celebrities to just shut th...,https://nypost.com/2026/01/26/us-news/dems-sho...
4,2026-01-26-13-03-25 +0000,wapo,Live updates: Court to hold hearings in Minnes...,https://www.washingtonpost.com/nation/2026/01/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2447/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
74,trump,33
48,shooting,25
85,minneapolis,21
40,minnesota,15
6,storm,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
151,2026-01-26-01-57-16 +0000,nypost,Trump ‘reviewing everything’ about fatal shoot...,https://nypost.com/2026/01/25/us-news/trump-re...,130
25,2026-01-26-12-22-24 +0000,nypost,Powerful House Republican suggests Trump consi...,https://nypost.com/2026/01/26/us-news/powerful...,123
96,2026-01-26-09-46-21 +0000,bbc,Trump says administration 'reviewing everythin...,https://www.bbc.com/news/articles/cr571qg4m61o...,98
32,2026-01-26-12-03-00 +0000,wsj,Republican unity over President Trump’s immigr...,https://www.wsj.com/politics/policy/miller-noe...,94
184,2026-01-25-22-29-48 +0000,bbc,NRA and other US gun activists push back at Tr...,https://www.bbc.com/news/articles/cnvg812n01no...,87


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
151,130,2026-01-26-01-57-16 +0000,nypost,Trump ‘reviewing everything’ about fatal shoot...,https://nypost.com/2026/01/25/us-news/trump-re...
109,50,2026-01-26-07-47-57 +0000,nypost,Missing Kansas teacher found dead in snow afte...,https://nypost.com/2026/01/26/us-news/missing-...
138,50,2026-01-26-04-30-19 +0000,nypost,Violent anti-ICE protesters swarm Minnesota ho...,https://nypost.com/2026/01/25/us-news/anti-ice...
178,38,2026-01-25-22-48-00 +0000,wsj,Senate Democrats angered by the deadly shootin...,https://www.wsj.com/politics/policy/senate-dem...
104,36,2026-01-26-08-36-39 +0000,wsj,"Stock Market Today: Gold Price Tops $5,000 for...",https://www.wsj.com/livecoverage/stock-market-...
15,32,2026-01-26-12-46-55 +0000,nyt,"In Xi’s China, Top General’s Fall Shows Precar...",https://www.nytimes.com/2026/01/26/world/asia/...
129,27,2026-01-26-05-06-32 +0000,nypost,Crazy scheme saw San Francisco taxpayers shell...,https://nypost.com/2026/01/26/us-news/san-fran...
3,26,2026-01-26-13-04-50 +0000,nypost,Dems need to ‘tell celebrities to just shut th...,https://nypost.com/2026/01/26/us-news/dems-sho...
32,26,2026-01-26-12-03-00 +0000,wsj,Republican unity over President Trump’s immigr...,https://www.wsj.com/politics/policy/miller-noe...
147,25,2026-01-26-03-05-19 +0000,nypost,Private plane carrying 8 passengers crashes in...,https://nypost.com/2026/01/25/us-news/private-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
